In [12]:
# inporting our own functions
from modules.TrainTestValGen import init_gens
# @TODO add more to the modules folder to save space 

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pickle
import gc
from itertools import chain
from _thread import start_new_thread

import tensorflow as tf
from keras import backend as K
from keras import regularizers, optimizers

from keras.utils import to_categorical
from keras_preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dense, Conv2D, Flatten, Activation, Dropout, MaxPooling2D, BatchNormalization

from keras.backend.tensorflow_backend import set_session
from keras.backend.tensorflow_backend import clear_session
from keras.backend.tensorflow_backend import get_session

C:\Users\saber\Anaconda3\envs\tensorflow\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\saber\Anaconda3\envs\tensorflow\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\saber\Anaconda3\envs\tensorflow\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users\saber\Anaconda3\envs\tensorflow\lib\site-packages

In [2]:
# @TODO Get this working an import

def LabelPicker(label='Cardiomegaly'):
    global train_df
    global test_df
    global df
    df = de_df[de_df['Finding Labels'] == label]
    
    df = pd.concat([df,NoF_df])
    
    train_df = pd.merge(train_filter,df , on='Image Index', how='inner')
    test_df = pd.merge(test_filter,df , on='Image Index', how='inner')
    

#Attach CSV to Images Folder and defline our traiing validation and test
def vaild_gen(batch_fit, Class_Value, TARGET_SIZE):
    global valid_generator
    valid_generator = datagen.flow_from_dataframe(
        dataframe=train_df,
        directory="../images/",
        x_col="Image Index",
        y_col=label,
        subset="validation",
        class_mode=Class_Value,
        target_size=(TARGET_SIZE,TARGET_SIZE),
        batch_size=batch_fit)
    
def train_gen(batch_fit, Class_Value, TARGET_SIZE):
    global train_generator
    train_generator=datagen.flow_from_dataframe(
        dataframe=train_df,
        directory="../images/",
        x_col="Image Index",
        y_col=label,
        subset="training",
        class_mode=Class_Value,
        target_size=(TARGET_SIZE,TARGET_SIZE),
        batch_size=batch_fit)
    
def test_gen(batch_fit, Class_Value, TARGET_SIZE):
    global test_generator
    test_generator=test_datagen.flow_from_dataframe(
        dataframe=test_df,
        directory="../images/",
        x_col="Image Index",
        y_col=label,
        class_mode=Class_Value,
        target_size=(TARGET_SIZE,TARGET_SIZE),
        batch_size=batch_fit)
    
def init_gens(batch_fit, class_value, target):
    start_new_thread(vaild_gen,(batch_fit, class_value,target,  ))
    start_new_thread(train_gen,(batch_fit, class_value,target, ))
    start_new_thread(test_gen,(batch_fit, class_value,target, ))
    
# Resetting global Memory https://github.com/keras-team/keras/issues/12625
# TODO move to module
def reset_keras():
    sess = get_session()
    clear_session()
    sess.close()
    sess = get_session()

    try:
        del classifier # this is from global space - change this as you need
    except:
        pass
    # use the same config as you used to create the session
    config = tf.ConfigProto()
    config.gpu_options.per_process_gpu_memory_fraction = 1
    config.gpu_options.visible_device_list = "0"
    set_session(tf.Session(config=config))
    

In [9]:
# Load data
BBox_df = pd.read_csv('../BBox_List_2017.csv',dtype=str)
de_df = pd.read_csv('../Data_Entry_2017.csv',dtype=str)
train_filter = pd.read_csv('../train_val_list.txt',dtype=str)
train_filter.columns=['Image Index']
test_filter = pd.read_csv('../test_list.txt',dtype=str)
test_filter.columns=['Image Index']

In [10]:
bad_ages = de_df[de_df['Patient Age'].astype(int) >= 110].index

de_df = de_df.drop(bad_ages)

de_df['Finding Labels'] = de_df['Finding Labels'].map(lambda x: x.replace('No Finding', ''))

all_labels = np.unique(list(chain(*de_df['Finding Labels'].map(lambda x: x.split('|')).tolist())))

all_labels = [x for x in all_labels if len(x) > 0]

# print('All Labels ({}): {}'.format(len(all_labels), all_labels))

for c_label in all_labels:
    if len(c_label)>1: # leave out empty labels
        de_df[c_label] = de_df['Finding Labels'].map(lambda finding: '1' if c_label in finding else '0')

de_df = de_df.drop(columns=['Unnamed: 11','Patient Gender','View Position','Patient Age','Patient ID','Follow-up #','OriginalImagePixelSpacing[x','y]','OriginalImage[Width','Height]'])

NoF_df = de_df[de_df['Finding Labels'] == '' ].sample(30000)


In [11]:
# 1/255 adjust the color
datagen = ImageDataGenerator(rescale=1./255 ,validation_split=0.25)
test_datagen = ImageDataGenerator(rescale=1./255)

# To denote what label to test on
label = 'Cardiomegaly'

LabelPicker(label)

# Class Mode
Class_Value = 'categorical'

# Target_size for all
TARGET_SIZE = 270

# How we bin out the batches to run onto the GPU
BATCH_SIZE = 32

In [12]:
init_gens(BATCH_SIZE ,Class_Value, TARGET_SIZE)

C:\Users\saber\Anaconda3\envs\tensorflow\lib\site-packages\keras_preprocessing\image\dataframe_iterator.py:273: UserWarning: Found 336 invalid image filename(s) in x_col="Image Index". These filename(s) will be ignored.
  .format(n_invalid, x_col)


Found 4817 validated image filenames belonging to 2 classes.


C:\Users\saber\Anaconda3\envs\tensorflow\lib\site-packages\keras_preprocessing\image\dataframe_iterator.py:273: UserWarning: Found 2419 invalid image filename(s) in x_col="Image Index". These filename(s) will be ignored.
  .format(n_invalid, x_col)


Found 5880 validated image filenames belonging to 2 classes.
Found 17640 validated image filenames belonging to 2 classes.


In [14]:
# @TODO read https://medium.com/nanonets/how-to-easily-build-a-dog-breed-image-classification-model-2fd214419cde

model = Sequential()
model.add(Conv2D(32, (3, 3), padding='same',
                 input_shape=(TARGET_SIZE,TARGET_SIZE,3)))
model.add(Activation('relu'))

model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))

model.add(MaxPooling2D(pool_size=(2, 2)))


model.add(Conv2D(128, (3, 3), padding='same'))
model.add(Activation('relu'))

model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Activation('relu'))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(256))
model.add(Activation('relu'))
model.add(Dropout(0.5))

model.add(Dense(2, activation='softmax'))

In [15]:
STEP_SIZE_VALID= valid_generator.n//valid_generator.batch_size

STEP_SIZE_TRAIN= train_generator.n//train_generator.batch_size

STEP_SIZE_TEST=test_generator.n//test_generator.batch_size


model.compile(optimizers.rmsprop(lr=0.0001),
loss="categorical_crossentropy", metrics=["accuracy"])


model.fit_generator(generator=train_generator,
                    steps_per_epoch=STEP_SIZE_TRAIN,
                    validation_data=valid_generator,
                    validation_steps=STEP_SIZE_VALID,
                    workers=16,
                    use_multiprocessing=False,
                    epochs=1) 

W0803 00:09:27.668281 22824 deprecation_wrapper.py:119] From C:\Users\saber\Anaconda3\envs\tensorflow\lib\site-packages\keras\optimizers.py:790: The name tf.train.Optimizer is deprecated. Please use tf.compat.v1.train.Optimizer instead.

W0803 00:09:27.672282 22824 deprecation_wrapper.py:119] From C:\Users\saber\Anaconda3\envs\tensorflow\lib\site-packages\keras\backend\tensorflow_backend.py:3295: The name tf.log is deprecated. Please use tf.math.log instead.

W0803 00:09:27.723294 22824 deprecation.py:323] From C:\Users\saber\Anaconda3\envs\tensorflow\lib\site-packages\tensorflow\python\ops\math_grad.py:1250: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Epoch 1/1
551/551 [==============================] - 104s 188ms/step - loss: 0.4693 - acc: 0.9701 - val_loss: 0.5642 - val_acc: 0.9650


In [17]:
model.evaluate()

ValueError: If evaluating from data tensors, you should specify the `steps` argument.

In [19]:
test_generator.head()

AttributeError: 'DataFrameIterator' object has no attribute 'head'